In [1]:
import numpy as np
from numba import jit
import xarray as xr
import matplotlib.pyplot as plt

#### Code for a moist energy balance model (MEBM) and CMIP output
The model solves for an anomalous zonal-mean profile of $T$ and $P-E$ assuming atmospheric heat transport is represented as down-gradient diffusion of near-surface moist-static energy. A simple Hadley Cell parameterization is then used to accurately simulate latent and dry-static energy transport. The model is based on Roe et al., (2015); Bonan et al., (2018), and Siler et al., (2018).

Dave Bonan (5/15/2022)

In [2]:
# Parameters

# number of grid points
l   = 180

# sine of latitude
x   = np.linspace(-1, 1., l )
dx  = x[1] - x[0]
lat = np.arcsin(x) * 180. / np.pi
nyt = len(x)

lv  = 2.5e6   # latent heat of vaporization
cp  = 1005    # heat capacity of air
ps  = 9.8e4   # Surface pressure
g   = 9.81    # Gravitational acceleration
D   = 1.16e6  # Diffusivity
a   = 6.371e6 # Radius of Earth

cw  = 6*60*60*24*365.25

In [ ]:
# import unperturbed climate
file       = xr.open_dataset('netCDF_files/CMIP_MEBM_control.nc')
lat        = np.array(file['lat'])
x          = np.array(file['x'])
T_clim     = np.array(file['T_clim'])
divFL_clim = np.array(file['divFL_clim'])
psi_clim   = np.array(file['psi_clim'])
gms_clim   = np.array(file['gms_clim'])

# import forcing variables
file       = xr.open_dataset('netCDF_files/MEBM_abrupt4xCO2_input.nc')
B          = np.array(file['B'])
G          = np.array(file['G'])
R          = np.array(file['R'])

In [ ]:
# hydrologic parameters
sigma      = 0.3
expon      = 2
w          = 1-np.exp(-(abs(x)/sigma)**expon)
EQ         = np.where(lat>=0)[0][0]
gms_factor = 1.08

In [ ]:
@jit(nopython=True)
def adams_bashforth(old,ddt,dt):
    """
    Computes the numerical solution of an ordinary differential equation 
    using a linear multistep method (Adams-Bashforth).             
    """
    new = old + dt * ( 23*ddt[-1] - 16*ddt[-2] + 5*ddt[-3])/12
    
    return new

@jit(nopython=True)
def qs(T):
    """
    Computes near-surface specific humidity (assuming fixed relative 
    humidity at 80%).
    """

    relhum  = 0.800
    epsilon = 0.622   
    psfc    = 9.8e4 
    e0      = 611.2
    a       = 17.67
    b       = 243.5

    qsat = epsilon*relhum/psfc*e0*np.exp(a*(T-273.15)/(b+T-273.15));
    
    return qsat

@jit(nopython=True)
def mebm_perturbation(T,B,R,G,t_ctrl,q_ctrl,psi_ctrl,gms_ctrl):
    """
    The perturbed moist energy balance model which assumes F is proportional 
    to h = c_p*T+L_v*qs(T).
    """

    # near-surface moist static energy assuming RH=80%
    q    = qs(T+t_ctrl)
    hs   = cp*(T)
    hl   = lv*(q-q_ctrl)
    h    = hs+hl
    h_e  = h/cp

    divF    = np.zeros((nyt))
    divFL   = np.zeros((nyt))
    F       = np.zeros((nyt))
    FL_EDDY = np.zeros((nyt))
    FS_EDDY = np.zeros((nyt))

    # solve for anomalous atmospheric heat transport
    for i in range(1, nyt-1):
        divF[i] = (cp*D*ps/(a**2*g) / dx**2) * (1. - x[i]**2) * (h_e[i+1] - 2 * h_e[i] + h_e[i-1]) - (cp*D*ps/(a**2*g) * x[i] / dx) * (h_e[i+1] - h_e[i-1])

    divF[ 0] = -cp*D*ps/(a**2*g) * 2 * x[0] * (h_e[ 1] - h_e[ 0]) / dx
    divF[-1] = -cp*D*ps/(a**2*g) * 2 * x[-1] * x[-1] * (h_e[-1] - h_e[-2]) / dx 

    # solve for E-P with Hadley cell weighting function
    for i in range(0, nyt):
        if i == nyt-1:
            F[i]           = -2*np.pi*ps/g*D*(1-x[i]**2)*(1/2*h[i-2] - 2*h[i-1] + 3/2*h[i])/dx
            FL_EDDY[i]     = -2*np.pi*ps/g*D*(1-x[i]**2)*w[i]*(-3/2*hl[i] + 2*hl[i-1] - 1/2*hl[i-2])/dx
            FS_EDDY[i]     = -2*np.pi*ps/g*D*(1-x[i]**2)*w[i]*(-3/2*hs[i] + 2*hs[i-1] - 1/2*hs[i-2])/dx

        elif i == 0:
            F[i]           = -2*np.pi*ps/g*D*(1-x[i]**2)*(-3/2*h[i] + 2*h[i+1] - 1/2*h[i+2])/dx
            FL_EDDY[i]     = -2*np.pi*ps/g*D*(1-x[i]**2)*w[i]*(-3/2*hl[i] + 2*hl[i+1] - 1/2*hl[i+2])/dx
            FS_EDDY[i]     = -2*np.pi*ps/g*D*(1-x[i]**2)*w[i]*(-3/2*hs[i] + 2*hs[i+1] - 1/2*hs[i+2])/dx

        else:
            F[i]           = -2*np.pi*ps/g*D*(1-x[i]**2)*0.5*((h[i]+h[i+1]) - (h[i]+h[i-1]))/dx
            FL_EDDY[i]     = -2*np.pi*ps/g*D*(1-x[i]**2)*0.5*w[i]*((hl[i]+hl[i+1]) - (hl[i]+hl[i-1]))/dx
            FS_EDDY[i]     = -2*np.pi*ps/g*D*(1-x[i]**2)*0.5*w[i]*((hs[i]+hs[i+1]) - (hs[i]+hs[i-1]))/dx

    F_HC = F-FL_EDDY-FS_EDDY
    
    gms = -h+(h[EQ]+h[EQ-1])/2*gms_factor
    
    psi = (F_HC-psi_ctrl*gms)/(gms_ctrl+gms)
    
    FL_HC=-lv*(psi_ctrl*(q-q_ctrl)+psi*q_ctrl)

    FL=FL_EDDY+FL_HC

    for i in range(0, nyt):
            if i == nyt-1:
                divFL[i]     = (1/(2*np.pi*a**2))*(1/2*FL[i-2] - 2*FL[i-1] + 3/2*FL[i])/dx
            elif i == 0:
                divFL[i]     = (1/(2*np.pi*a**2))*(-3/2*FL[i] + 2*FL[i+1] - 1/2*FL[i+2])/dx
            else:
                divFL[i]     = (1/(2*np.pi*a**2))*0.5*((FL[i]+FL[i+1]) - (FL[i]+FL[i-1]))/dx
    
    return T,divF,divFL,F,FL,FS_EDDY,FL_EDDY,FL_HC,psi,gms

In [ ]:
for mod in range(1):
    print('Running MEBM...')
    Bn=B
    Gn=G
    Rn=R
    t_ctrl = T_clim
    q_ctrl = qs(t_ctrl)
    psi_ctrl = psi_clim
    gms_ctrl = gms_clim
    dt  = 2500
    dT  = 86400*30
    nyear = 70
    nstep = int(nyear*360*86400/dt)
    nOut  = int(nyear*360*86400/dT)

    ddT_y    = np.zeros((3, nyt))
    TT_y     = np.zeros((2, nyt))

    ky     = 0

    TT_y[0]  = np.zeros((nyt))

    for i in range(0, nstep):

        ctime = i*dt

        if np.mod(ctime, dT)<dt:

            if ky % 100 == 0:
                print(f"Working on Month = {ky}, T' (Global Mean) = {np.nanmean(TT_y[0]):.1f}")
            elif np.isnan(TT_y[0].max()):
                break 
            ky      += 1

        if i < 2:

            T,divF,divFL,F,FL,FS_EDDY,FL_EDDY,FL_HC,psi,gms  = mebm_perturbation(TT_y[0],Bn,Rn,Gn,t_ctrl,q_ctrl,psi_ctrl,gms_ctrl)
            rhs_ts = (Rn-Gn+(Bn*T) +divF) / cw

            ddT_y[i] = rhs_ts

            TT_y[1]  = TT_y[0] + dt * ddT_y[i]

        else:

            T,divF,divFL,F,FL,FS_EDDY,FL_EDDY,FL_HC,psi,gms  = mebm_perturbation(TT_y[0],Bn,Rn,G,t_ctrl,q_ctrl,psi_ctrl,gms_ctrl)


            rhs_ts = (Rn-Gn+(Bn*T) + divF) / cw

            ddT_y[-1] = rhs_ts

            TT_y[1]   = adams_bashforth(TT_y[0], ddT_y,dt)

            ddT_y     = np.roll(ddT_y, 2, axis=0)

        TT_y = np.roll(TT_y, 1, axis=0)

In [ ]:
plt.figure(figsize=(10,10))
plt.subplot(4,3,1)
plt.plot(lat,Rn)
plt.xlabel('Latitude')
plt.ylabel('W m$^{-2}$')

plt.subplot(4,3,2)
plt.plot(lat,-Gn)
plt.xlabel('Latitude')
plt.ylabel('W m$^{-2}$')

plt.subplot(4,3,3)
plt.plot(lat,Bn)
plt.xlabel('Latitude')
plt.ylabel('W m$^{-2}$ K$^{-1}$')

plt.subplot(4,3,4)
plt.plot(lat,divF)
plt.xlabel('Latitude')
plt.ylabel('W m$^{-2}$')

plt.subplot(4,3,5)
plt.plot(lat,T)
plt.xlabel('Latitude')
plt.ylabel('K')

plt.subplot(4,3,6)
plt.plot(lat,divFL)
plt.xlabel('Latitude')
plt.ylabel('W m$^{-2}$')

plt.subplot(4,3,7)
plt.plot(lat,F,label='F')
plt.plot(lat,FL,ls='--',c='C0',label='$F_L$')
plt.plot(lat,F-FL,ls=':',c='C0',label='$F_S$')
plt.legend()
plt.xlabel('Latitude')
plt.ylabel('W')

plt.subplot(4,3,8)
plt.plot(lat,FL,label='$F_L$')
plt.plot(lat,FL_EDDY,ls='--',c='C0',label='$F_{L,{\mathrm{EDDY}}}$')
plt.plot(lat,FL_HC,ls=':',c='C0',label='$F_{L,{\mathrm{HC}}}$')
plt.legend()
plt.xlabel('Latitude')
plt.ylabel('W')

plt.subplot(4,3,9)
plt.plot(lat,psi)
plt.xlabel('Latitude')
plt.ylabel('kg s$^{-1}$')

plt.subplot(4,3,10)
plt.plot(lat,gms)
plt.xlabel('Latitude')
plt.ylabel('J kg$^{-1}$')

plt.tight_layout()

In [ ]:
from netCDF4 import Dataset 
from numpy import dtype
ncout = Dataset('netCDF_files/MEBM_perturbation_output.nc', 'w', format='NETCDF4')

ncout.createDimension('nyt',180)

vout1  = ncout.createVariable('lat'     , dtype('double').char,('nyt'))
vout2  = ncout.createVariable('x'       , dtype('double').char,('nyt'))
vout3  = ncout.createVariable('T'       , dtype('double').char,('nyt'))
vout4  = ncout.createVariable('divF'    , dtype('double').char,('nyt'))
vout5  = ncout.createVariable('divFL'   , dtype('double').char,('nyt'))
vout6  = ncout.createVariable('F'       , dtype('double').char,('nyt'))
vout7  = ncout.createVariable('FL'      , dtype('double').char,('nyt'))
vout8  = ncout.createVariable('FS_EDDY' , dtype('double').char,('nyt'))
vout9  = ncout.createVariable('FL_EDDY' , dtype('double').char,('nyt'))
vout10 = ncout.createVariable('FL_HC'   , dtype('double').char,('nyt'))
vout11 = ncout.createVariable('psi'     , dtype('double').char,('nyt'))
vout12 = ncout.createVariable('gms'     , dtype('double').char,('nyt'))

vout1[:]  = lat
vout2[:]  = x
vout3[:]  = T
vout4[:]  = divF 
vout5[:]  = divFL 
vout6[:]  = F 
vout7[:]  = FL 
vout8[:]  = FS_EDDY
vout9[:]  = FL_EDDY 
vout10[:] = FL_HC 
vout11[:] = psi 
vout12[:] = gms 

ncout.close()